# Human Activity Recognition (HAR)
사람은 일상생활에서 다양한 행동을 수행하며, 이러한 행동 패턴은 그 사람의 의도, 습관, 심리 상태 등을 반영할 수 있습니다. 이러한 이유로, 인지과학에서 헬스케어까지 여러 연구 분야에서 센서 데이터를 이용해 사람의 행동을 인식하려는 Human Activity Recognition (HAR) 연구가 활발히 이루어지고 있습니다.

이번 실습에서는 웨어러블 기기에서 얻은 시계열 센서 데이터를 활용하여 사람의 행동을 예측하는 딥러닝 모델을 학습해 보겠습니다.

## RealWorld HAR dataset 

HAR 연구를 위한 데이터셋은 단순한 행동(예: 걷기, 뛰기) 예측부터 정서나 의도 예측에 이르기까지 매우 다양합니다.

이번 실습에서는 스마트워치와 같은 손목에 착용한 가속도 센서로부터 수집된 데이터를 사용하여 HAR를 수행합니다.

[ReadWorld HAR 데이터셋](https://ieeexplore.ieee.org/document/7456521)
- 15명의 사람의 가속도계(acceleration), GPS, 자이로센서(gyroscope), 조도센서(light), 자기장(magnetic field), 사운드 레벨 센서(sound level) 데이터를 포함하고 있습니다.
- 센서는 여러 신체 부위에 부착되었습니다 (chest, forearm, head, shin, thigh, upper arm, and waist). 
- 각각의 사람은 8개의 활동 (climbing stairs up and down, jumping, lying, running, jogging, sitting, standing, walking)을 약 10분씩 수행하였습니다 (점프는 약 1.7분 수행)

이번 실습에서는 이 데이터셋 중 손목에 착용한 가속도계 센서 데이터만을 사용하여 HAR를 수행해 보겠습니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
csv_file_name = "/datasets/RealWorldHAR/forearm_acc.csv"
data = pd.read_csv(csv_file_name, header = 0)

### 데이터셋 살펴보기

In [ ]:
data.head(10)

In [ ]:
data.shape

각 행은 특정 참가자(subject_id)가 특정 활동(activity_label)을 수행하는 동안 손목에 착용한 센서로 부터 수집된 측정 값을 포함하고 있다.

센서 데이터는 50Hz의 샘플링 속도로 수집되었으며, 각 측정값은 여러 행에 걸쳐 저장되어 있습니다. 따라서 동일한 subject_id와 activity_label을 가진 연속적인 50개 행의 데이터는 1초 동안의 측정값들을 나타냅니다.

* subject_id: 데이터가 어떤 사람에게서 수집되었는지를 식별하는 고유 식별자
* timestamp : timestamp: 
* acc_x: 손목에서 얻은 x축 방향의 가속도 데이터
* acc_y: 손목에서 얻은 y축 방향의 가속도 데이터
* acc_z: 손목에서 얻은 z축 방향의 가속도 데이터
* activity_label: 사람이 수행한 활동 명

## 데이터 시각화

<mark>실습</mark> 주어진 컬럼 이름 `column_name`에 대응하는 데이터 값들의 종류와 분포를 분석하기 위한 함수 `analyze_column`을 완성하세요
1. `unique_values`: pandas `unique()`함수를 이용하여 `column_name`에 해당하는 데이터 값들의 고유한 종류를 얻습니다.
2. `value_distribution`: pandas `value_counts()`함수를 이용하여 각 고유한 값에 대한 데이터 개수를 계산합니다.

In [ ]:
def analyze_column(df, column_name):
    """
    Analyzes the unique values and distribution of a specified column in the DataFrame.

    Parameters:
    df (pandas.DataFrame): The input DataFrame containing the data to be analyzed.
    column_name (str): The name of the column to analyze within the DataFrame.

    Returns:
    tuple: A tuple containing two elements:
        - unique_values (numpy.ndarray): An array of unique values in the specified column.
        - value_distribution (pandas.Series): A Series representing the distribution (counts) of each unique value in the column.

    Raises:
    ValueError: If the specified column does not exist in the DataFrame.

    Example:
    unique_activity_labels, activity_label_distribution = analyze_column(df, 'activity_label')
    """
    unique_values = data[column_name].unique()
    value_distribution = data[column_name].value_counts()

    return unique_values, value_distribution

In [ ]:
unique_labels, label_distribution = analyze_column(data, 'activity_label')
print(f'Unique labels in the dataset: {unique_labels}')
print(f'Label Distribution: {label_distribution}')

unique_subjects, subjects_distribution = analyze_column(data, 'subject_id')
print(f'\nSubject ID Distribution: {subjects_distribution}')

In [ ]:
def plot_label_dist(label_distribution):
    x = label_distribution.index.tolist()
    y = label_distribution.tolist()
    
    plt.figure(figsize=(12, 5))
    plt.bar(x, y, width=0.5)
    plt.xlabel('Activity label')
    plt.ylabel('Count')
    plt.title('Label Distribution')
    plt.show()

plot_label_dist(label_distribution)

이제 센서 데이터 값을 한번 시각화해보겠습니다. 

가속도 센서는 3차원 공간에서의 가속도 값과 방향을 측정합니다. 그러나 센서를 착용한 위치가 사람마다 다를 수 있어, 데이터 분석을 어렵게 합니다.

센서 데이터 분석에서 중요한 요소 중 하나는 샘플링 속도(sampling rate) (예: 50 Hz)와 센서의 측정 범위(range) (예: -8g에서 8g)입니다.

그외의 다른 요소들 (감도, 오프셋, 온도 교정 등)은 대부분의 경우 무시해도 괜찮습니다.

In [ ]:
def filter_df(df, target_label, target_subject):
    filtered_df = df[df["activity_label"].isin(target_label) & df["subject_id"].isin(target_subject)]
    return filtered_df

def plot_sensor_data(df, sensor_names, sampling_rate):
    subjects = df['subject_id'].unique()
    activities = df['activity_label'].unique()
    
    # Define grid size
    n_rows = len(activities)
    n_cols = len(subjects)
    
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols * 4, n_rows * 3), sharex=True, sharey=True)
    fig.suptitle('Sensor Data for Each Subject and Activity', y=1.02, fontsize=16)
    
    for i, activity in enumerate(activities):
        for j, subject in enumerate(subjects):
            # Filter data for current subject and activity
            subject_activity_df = df[(df['subject_id'] == subject) & (df['activity_label'] == activity)]
            
            if subject_activity_df.empty:
                continue
            
            x_axis = np.array(range(len(subject_activity_df))) / sampling_rate
            y_axis = subject_activity_df[sensor_names]
            
            # Select appropriate subplot
            if n_rows > 1 and n_cols > 1:
                ax = axs[i, j]
            elif n_rows > 1:
                ax = axs[i]
            else:
                ax = axs[j]
                
            for sensor in sensor_names:
                ax.plot(x_axis, y_axis[sensor], label=sensor)
            
            ax.set_title(f'Subject: {subject}, Activity: {activity}')
            ax.set_xlabel('Time (seconds)')
            ax.set_ylabel('Acceleration (10g)')
            ax.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

In [ ]:
filterd_data = filter_df(data, target_label=['walking', 'running', 'sitting', 'standing', 'climbing_up'], target_subject= [2, 3, 9])
plot_sensor_data(filterd_data, sensor_names = ['acc_x', 'acc_y', 'acc_z'], sampling_rate = 50)

<mark>질문</mark>: 모든 사람이 센서를 동일한 위치에 착용한 것 같나요?

## 데이터 전처리
pandas는 데이터 시각화나 EDA작업에서는 적합하지만 deep learning학습에서는 속도가 느려 적합하지 않습니다.

지금부터는 numpy를 사용하여 데이터를 다뤄 보겠습니다.

In [ ]:
def load_rwhar_data(csv_file_name):
    class_names = ['climbing_down', 'climbing_up', 'jumping', 'lying', 'running', 'sitting', 'standing', 'walking']
    sampling_rate = 50

    df = pd.read_csv(csv_file_name, sep=',', header=0)

    label_to_int = {label: idx for idx, label in enumerate(class_names)}

    X = df[["acc_x", "acc_y", "acc_z"]].to_numpy()
    subject_ids = df["subject_id"].to_numpy()
    y = df['activity_label'].map(label_to_int).to_numpy()
    
    return X, y, subject_ids, class_names, sampling_rate

In [ ]:
X, y, subject_ids, class_names, sampling_rate = load_rwhar_data(csv_file_name)

print(X.shape, y.shape, subject_ids.shape)

- `X`는 머신러닝 모델의 입력으로 acc_x, acc_y, acc_z 센서 값들을 저장하고 있다
- `y`는 머신러닝 모델의 target (ground truth)으로 0~7의 정수로 encoding된 'activity_label'을 저장하고 있다.
- `subject_ids`에는 'X', 'y' 데이터에 대응하는 subject_id를 저장하고 있다.

### missing value interpolation

센서 데이터에서는 배터리 문제, 센서 결함, 통신 장애 등으로 인해 missing value가 발생하는 경우가 빈번합니다. 

이러한 결측값을 처리하기 위해 보간(interpolation) 기법이 널리 사용됩니다.

가장 단순한 방법 중 하나는 <b>선형 보간 (linear interpolation)</b>으로, 연속된 데이터 포인트 간에 직선을 그어 결측값을 추정하는 방식입니다. 

그러나 데이터의 패턴이 더 복잡하거나 비선형적인 경우에는 KNN 기반 기법, 딥러닝 기반 모델 등 더 고급 기법들이 사용될 수 있습니다. 이러한 다양한 보간 방법을 통해 센서 데이터의 연속성과 품질을 효과적으로 향상시킬 수 있습니다.

In [ ]:
import random
X_with_nan = X.copy()

# Simulate missing data
random.seed(0)
for i in range(0, 10):
    fill_index = random.randint(1, 20)
    X_with_nan[fill_index] = [np.nan, np.nan, np.nan]

print(X_with_nan[:20])

In [ ]:
def interpolate_nan_values(X):
    # Convert to DataFrame
    df = pd.DataFrame(X)

    # Interpolate NaN values
    df_interpolated = df.interpolate(method='linear', axis=0, limit_direction='both')

    # Convert back to numpy array
    X_interpolated = df_interpolated.values
    return(X_interpolated)


In [ ]:
X_interpolated= interpolate_nan_values(X_with_nan)
print(X_interpolated[:20])

### Resampling
만약 센서 데이터간의 sampling rate가 서로 다르다면 동일한 sampling rate로 다시 샘플링하는 과정이 필요합니다.

이는 한 데이터의 샘플링 레이트에 맞춰 다른 데이터를 <b>업샘플링(up-sampling)</b>하거나 <b>다운샘플링(down-sampling)</b>하는 방식으로 수행됩니다.

In [ ]:
def resample_data(data, freq_old, freq_new):
    """
    Function which changes the sampling rate of some data via interpolation

    Parameters
        data (numpy array): Data to be resampled
        freq_old (int): Old sampling rate
        freq_new (int) : New sampling rate

    Returns: numpy array
        Resampled data
    """
    t_old = np.arange(data.shape[0]) / freq_old
    duration = t_old[-1]

    num_samples = int(np.floor(duration * freq_new)) + 1
    t_new = np.linspace(t_old[0], t_old[-1], num_samples)

    dataNew = np.interp(t_new, t_old, data)

    return t_new, dataNew

In [ ]:
X_sensor1 = X[:100, 0]
freq_old = 50
freq_new = 35

t_new, X_sensor1_resampled = resample_data(X_sensor1, freq_old, freq_new)

print(f"Original sensor data shape: {X_sensor1.shape}")
print(f"Resampled sensor data shape: {X_sensor1_resampled.shape}")

In [ ]:
t_old = np.arange(X_sensor1.shape[0]) / freq_old
plt.figure(figsize=(10, 4))
plt.plot(t_old, X_sensor1, label=f'Original data ({freq_old} Hz)', alpha=0.7)
plt.plot(t_new, X_sensor1_resampled, 'o-', label=f'Resampled data ({freq_new} Hz)', alpha=0.7)
plt.xlabel('Seconds')
plt.ylabel('Sensor value')
plt.legend()
plt.grid(True)
plt.show()

### Scaling
모든 머신러닝 및 딥러닝 모델의 입력 데이터는 스케일링(정규화) 과정을 거치는 것이 일반적입니다. 데이터 스케일링은 각 <b>feature(특징)</b>에 대해 개별적으로 수행됩니다. 

이를 통해 센서별로 측정값의 범위나 분포가 서로 다른 경우에도 모든 feature가 비슷한 분포를 가지도록 정규화할 수 있습니다.

예를 들어, 가속도계(accelerometer), 자이로스코프(gyroscope), 그리고 자기계(magnetometer) 센서는 서로 다른 단위 및 범위를 가질 수 있습니다. 

이러한 데이터들을 정규화하지 않고 학습에 사용하면 모델이 특정 feature에 더 높은 중요도를 부여할 수 있어 학습 성능이 저하될 수 있습니다.

In [ ]:
import matplotlib.patches as mpatches

def plot_data(data, title):
    """
    Function to plot data

    :param data: dataframe
        Data to be plotted
    :param title: string
        Title of plot
    :return:
    """
    indices = []

    for i in range(0, len(data)):
        indices.append(i)
    fig, axes = plt.subplots(1, 1, sharex=True, sharey=False, figsize=(14, 8))
    x_patch = mpatches.Patch(color='orange', label='x-axis')
    y_patch = mpatches.Patch(color='green', label='y-axis')
    z_patch = mpatches.Patch(color='blue', label='z-axis')
    plt.legend(handles=[x_patch, y_patch, z_patch])
    axes.plot(data, lw=2, ls='-')
    axes.set_ylabel('Acceleration (mg)', fontsize=12)
    plt.xlabel('Index', fontsize=16)
    plt.suptitle(title, fontsize=19)
    plt.tight_layout()
    fig.subplots_adjust(top=0.88)
    plt.show()

In [ ]:
plot_data(X[:100000], 'Original Dataset')

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1,1))

X_scaled = scaler.fit_transform(X)

plot_data(X_scaled[:100000], "Scaled")

print("Scaled:")
print("Mean(x): " + str(np.mean(X_scaled[:, 0])) + "; Std(x): " + str(np.std(X_scaled[:, 0])))
print("Mean(y): " + str(np.mean(X_scaled[:, 1])) + "; Std(y): " + str(np.std(X_scaled[:, 1])))
print("Mean(z): " + str(np.mean(X_scaled[:, 2])) + "; Std(z): " + str(np.std(X_scaled[:, 2])))


### Jumping/Sliding Window
시계열 데이터를 머신러닝 모델의 학습에 사용하기 위해서는 데이터를 더 작은 크기의 <b>윈도우(window)</b>로 나누는 과정이 필요합니다. 

이 과정은 슬라이딩 윈도우(sliding window) 알고리즘을 통해 수행되며, 고정된 크기의 윈도우가 이동하며 데이터를 나누게 됩니다.

이 알고리즘의 첫 번째 인자인 `window_size`는 매우 중요한 파라미터로, 데이터를 얼마나 큰 윈도우로 나눌지를 결정합니다.
- 만약 활동 간의 전환이 매우 빠른 데이터라면, `window_size`를 작게 설정하여 데이터를 더 작은 조각으로 나누는 것이 적절할 수 있습니다. 또한, window_size가 작을수록 더 많은 학습 데이터 조각을 얻을 수 있습니다.
- 그러나 `window_size`가 너무 작으면 데이터의 특징적인 패턴을 제대로 포착하지 못할 수 있으며, 결과적으로 데이터의 표현력이 감소할 수 있습니다.
- 따라서 슬라이딩 윈도우 크기를 결정하는것은 단순하지 않으며, 최종 모델 성능에 큰 영향을 미치는 중요한 요소입니다.

두 번째 인자인 `overlap_ratio`는 각 윈도우가 서로 얼마나 겹치는지를 정의합니다.
- <b>슬라이딩 윈도우의 겹침(overlap)</b>이 클수록, HAR 분류기의 성능이 향상된다고 알려져 있습니다. 이는 데이터의 양이 많아지고 활동 간의 전환과 같은 중요한 순간을 놓치지 않기 때문입니다.

<mark>실습</mark> sliding windows를 적용하는 함수 `sliding_window_samples`를 완성하세요

In [ ]:
def apply_sliding_window(X, y, window_size, overlap_ratio):
    """
    Generate sliding windows from X, y for a single dataset.
    
    Parameters:
    - X: The input data of shape (n_samples, n_features).
    - y: The target labels of shape (n_samples,).
    - window_size: The size of each window (in number of timesteps).
    - overlap_ratio: The proportion of overlap between consecutive windows (0 to 1).
    
    Returns:
    - X_windows: Numpy array of shape (n_windows, window_size, n_features) containing the sliding windows.
    - y_windows: Numpy array of shape (n_windows,) where each window's label is the last label in the window.
    """

    step_size = int(window_size * (1 - overlap_ratio))
    
    X_windows = []
    y_windows = []

    n_samples = len(X)
    for start in range(0, n_samples - window_size + 1, step_size):
        ##### YOUR CODE START #####  
        X_window = None # TODO
        y_window = None # TODO
        ##### YOUR CODE END #####

        X_windows.append(X_window)
        y_windows.append(y_window[-1]) ## Use the last label in the window for y

    X_windows = np.array(X_windows)
    y_windows = np.array(y_windows)

    return X_windows, y_windows


def apply_sliding_window_per_sample(X, y, subject_ids, window_size, overlap_ratio):
    assert overlap_ratio < 1
    X_all_windows = []
    y_all_windows = []
    subject_all_windows = []

    unique_subjects = np.unique(subject_ids)

    for subject in unique_subjects:
        subject_X = X[subject_ids == subject]
        subject_y = y[subject_ids == subject]

        X_windows, y_windows = apply_sliding_window(subject_X, subject_y, window_size, overlap_ratio)

        X_all_windows.append(X_windows)
        y_all_windows.append(y_windows)
        subject_all_windows.append(np.tile(subject, X_windows.shape[0]))

    X_all_windows = np.concatenate(X_all_windows, axis=0)
    y_all_windows = np.concatenate(y_all_windows, axis=0)
    subject_all_windows = np.concatenate(subject_all_windows, axis=0)

    return X_all_windows, y_all_windows, subject_all_windows

In [ ]:
X_windows, y_windows, subjectID_windows  = apply_sliding_window_per_sample(X, y, subject_ids, window_size = 50, overlap_ratio = 0.25)

In [ ]:
print("Original data shapes: \t\t\t", X.shape, y.shape, subject_ids.shape)
print("Data shapes after sliding windows: \t", X_windows.shape, y_windows.shape, subjectID_windows.shape)

**Expected output:**
```
Original data shapes: 			 (3200803, 3) (3200803,) (3200803,)
Data shapes after sliding windows: 	 (86495, 50, 3) (86495,) (86495,)
```


# Training a neural network

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset

from sklearn.preprocessing import MinMaxScaler

from training_utilities import create_dataloaders, train_loop, evaluation_loop, load_checkpoint, save_checkpoint

## DeepConvLSTM
이번 실습에서는 DeepConvLSTM이라는 아키텍처를 사용하여 사람 활동 인식(Human Activity Recognition, HAR)을 수행하는 딥러닝 모델을 학습해봅니다. ([논문 링크](https://www.mdpi.com/1424-8220/16/1/115))

구성 요소
1. 합성곱 층 (Convolutional Layers): 필터를 사용해 슬라이딩 윈도우 입력 데이터로 부터 특징을 추출합니다.

2. LSTM 층: 합성곱 층에서 추출된 특징들의 시간적(temporal) 패턴을 학습합니다.

3. 분류 층 (Classification Layer): LSTM 출력은 완전 연결층(fully connected layer)으로 전달되어 최종 예측을 생성합니다. 


In [ ]:
class DeepConvLSTM(nn.Module):
    def __init__(self, num_classes, config):
        super(DeepConvLSTM, self).__init__()
        # parameters
        self.num_classes = num_classes
        self.window_size = config['window_size']
        self.dropout_prob = config['dropout_prob']
        self.num_sensors = config['num_sensors']
        self.num_filters = config['num_filters']
        self.kernel_size = config['kernel_size']
        self.hidden_dim = config['hidden_dim']
        self.nb_layers_lstm = config['num_layers']

        # define activation function
        self.relu = nn.ReLU(inplace=True)

        # define conv layers
        self.conv1 = nn.Conv2d(1, self.num_filters, (self.kernel_size, 1))
        self.conv2 = nn.Conv2d(self.num_filters, self.num_filters, (self.kernel_size, 1))
        self.conv3 = nn.Conv2d(self.num_filters, self.num_filters, (self.kernel_size, 1))
        self.conv4 = nn.Conv2d(self.num_filters, self.num_filters, (self.kernel_size, 1))
        
        # define lstm layers
        self.lstm = nn.LSTM(input_size=self.num_filters * self.num_sensors, hidden_size=self.hidden_dim, num_layers=self.nb_layers_lstm)

        # define dropout layer
        self.dropout = nn.Dropout(self.dropout_prob)
        
        # define classifier
        self.fc = nn.Linear(self.hidden_dim, num_classes)

    def forward(self, x):
        # reshape data for convolutions
        x = x.view(-1, 1, self.window_size, self.num_sensors)
        
        # apply convolution and the activation function
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))  #(batch_size, num_filters, seq_length, num_sensors)
        
        # sets the final sequence length 
        final_seq_len = x.shape[2]
        
        # permute dimensions and reshape for LSTM
        x = x.permute(0, 2, 1, 3)
        x = x.reshape(-1, final_seq_len, self.num_filters * self.num_sensors) #(batch_size, seq_length, num_features)

        x, _ = self.lstm(x) #(batch_size, seq_length, hidden_dim)

        x = x[:, -1, :]  # shape (batch_size, hidden_dim)
        x = self.dropout(x)
        out = self.fc(x)
        
        return out

In [ ]:
def get_model(model_name, num_classes, config):
    if model_name == "DeepConvLSTM":
        model = DeepConvLSTM(num_classes, config)
    else:
        raise Exception("Model not supported: {}".format(model_name))
    
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"Using model {model_name} with {total_params} parameters ({trainable_params} trainable)")


    return model

## 데이터 준비하기

딥러닝 학습을 위한 데이터를 준비합니다. `load_RWHAR_datasets`함수는 아래의 과정을 거처 데이터를 준비합니다.
- 데이터 로드: CSV 파일로 부터 데이터를 로드합니다.
- 데이터 스케일링: MinMaxScaler를 사용해 X 데이터를 -1에서 1 범위로 스케일링 합니다.
- 슬라이딩 윈도우 적용: `apply_sliding_window_per_sample` 함수를 사용해 데이터를 슬라이딩 윈도우 방식으로 변환합니다.
- 데이터셋 분할: subject_id 를 기준으로 데이터를 train, validation, test set으로 나눕니다 (아래 설명 참고).
- 파이토치 데이터셋 정의: 분할된 데이터셋으로부터 데이터를 읽어오는 `RWHARDataset` 데이터셋을 정의합니다.
- 클래스 가중치 계산: `compute_class_weight`를 사용해 클래스 가중치를 계산합니다.

### 데이터셋 분할 (Splitting your data)

머신러닝 모델의 성능은 모델의 구조와 하이퍼파라미터 설정에 크게 의존합니다. 
모델 학습 과정에서 적절한 모델 구조와 하이퍼파라미터를 찾는 과정을 하이퍼파라미터 튜닝(hyperparameter tuning)이라고 합니다. 이 과정은 모델의 성능을 높이고, 새로운 데이터에 대해 잘 예측할 수 있도록 하는 데 핵심적인 역할을 합니다.

하이퍼파라미터가 적절한지 평가하기서는 위해 모델의 성능을 검증(validation) 데이터셋에서 확인합니다. 이를 위해 전체 데이터를 다음과 같이 세 부분으로 분할합니다:

- 훈련(train) set: 모델 학습에 사용됩니다.
- 검증(validation) set: 하이퍼파라미터 튜닝과 모델 선택에 사용됩니다.
- 테스트(test) set: 최종 모델의 성능을 평가하는 데 사용됩니다.

validation set 에서의 성능을 높이기 위해 하이퍼파라미터 튜닝을 반복하다 보면, 모델이 validation set에 과적합되어 새로운 데이터에 대한 일반화 성능이 떨어질 수 있습니다. 

이를 방지하기 위해 test set을 별도로 유지하여, 모델의 최종 성능을 객관적으로 평가합니다.

test set은 모델이 관찰하지 못한 새로운 데이터셋에서의 성능을 평가하기 위한 목적으로 모델 구조 개선이나 하이퍼파라미터 튜닝 과정에서 test set의 성능을 <b>절대로(!)</b> 참고해서는 안 됩니다. 

### 데이터셋 분할 방법

1. 임의 분할 (Random Split) : 데이터를 무작위로 섞은 후, 일정한 비율로 데이터셋을 나눕니다. 예를 들어, 70%는 훈련 세트로, 15%는 검증 세트로, 나머지 15%는 테스트 세트로 사용할 수 있습니다.

2. 참가자 기반 분할 (Subject-wise Split): 참가자에 따라 데이터셋을 나눕니다. 예를 들어, 10명의 참여자가 있다면 6명은 훈련 세트, 2명은 검증 세트, 2명은 테스트 세트로 사용할 수 있습니다. 이는 개인 간 변이를 고려하여 모델의 일반화 능력을 평가하는 데 유용합니다.

생물의학(biomedical) 데이터에서 <b>참가자 기반 분할은 데이터 누출(Data leakage)을 방지하는 데 필수적</b>입니다. 한 개인에게서 생성된 데이터간에는 서로 correlation이 존재한다. 따라서 같은 참여자의 데이터가 훈련 세트와 테스트 세트에 모두 포함되면, 모델이 개인의 특성을 학습하여 실제로는 일반화되지 않은 성능 향상을 보일 수 있습니다. 이는 모델이 새로운 환자나 참여자에게 적용될 때 성능이 저하되는 문제의 원인이 됩니다.

<mark>실습</mark> 참가자 ID (subject_id)를 기준으로 데이터를 분할하도록 `load_RWHAR_datasets` 함수를 완성하세요.
- `X_train`, `y_train` : subject_id <= 10
- `X_val`, `y_val` : (subject_id > 10) & (subject_id <= 12)
- `X_test`, `y_test` : subject_id > 12

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

def load_RWHAR_datasets(csv_file_name, config):
    X, y, subject_ids, class_names, sampling_rate = load_rwhar_data(csv_file_name)
    scaler = MinMaxScaler(feature_range=(-1,1))
    X_scaled = scaler.fit_transform(X)

    X_windows, y_windows, subjectID_windows  = apply_sliding_window_per_sample(X_scaled, y, subject_ids, window_size = config["window_size"], overlap_ratio = config["overlap_ratio"])

    ##### YOUR CODE START #####  
    X_train = None # TODO
    y_train = None # TODO
    X_val = None # TODO
    y_val = None # TODO
    X_test = None # TODO
    y_test = None # TODO
    ##### YOUR CODE END #####

    train_dataset = RWHARDataset(X_train, y_train, augment = True)
    val_dataset = RWHARDataset(X_val, y_val)
    test_dataset = RWHARDataset(X_test, y_test)

    class_weights = compute_class_weight(
        class_weight='balanced', 
        classes=np.unique(y_train), 
        y=y_train
    )
    class_weights = torch.tensor(class_weights, dtype=torch.float32)

    return train_dataset, val_dataset, test_dataset, class_names, class_weights

class RWHARDataset(Dataset):
    def __init__(self, X, y, augment=False):
        self.X = X
        self.y = y
        self.augment = augment

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X_sample = self.X[idx]
        y_sample = self.y[idx]

        if self.augment:
            X_sample = self.augment_data(X_sample)
        
        X_sample = torch.tensor(X_sample, dtype=torch.float32)
        y_sample = torch.tensor(y_sample, dtype=torch.long)
        
        return X_sample, y_sample
    
    def augment_data(self, X):
        if np.random.rand() > 0.5:
            X = X + 0.05 * np.random.randn(*X.shape)  # Add noise
        if np.random.rand() > 0.5:
            X = X * np.random.uniform(0.9, 1.1)  # Random scaling
        if np.random.rand() > 0.5:
            X = self.rotate_accelerometer_3d(X)
        
        return X
    
    def rotate_accelerometer_3d(self, X):
        """Randomly rotate accelerometer data"""
        theta_x = np.random.uniform(-np.pi / 10, np.pi / 10)
        theta_y = np.random.uniform(-np.pi / 10, np.pi / 10)
        theta_z = np.random.uniform(-np.pi / 10, np.pi / 10)

        rotation_matrix_x = np.array([
            [1, 0, 0],
            [0, np.cos(theta_x), -np.sin(theta_x)],
            [0, np.sin(theta_x), np.cos(theta_x)]
        ])

        rotation_matrix_y = np.array([
            [np.cos(theta_y), 0, np.sin(theta_y)],
            [0, 1, 0],
            [-np.sin(theta_y), 0, np.cos(theta_y)]
        ])

        rotation_matrix_z = np.array([
            [np.cos(theta_z), -np.sin(theta_z), 0],
            [np.sin(theta_z), np.cos(theta_z), 0],
            [0, 0, 1]
        ])

        rotation_matrix = np.dot(np.dot(rotation_matrix_x, rotation_matrix_y), rotation_matrix_z)
        X = np.dot(X, rotation_matrix) 

        return X

In [ ]:
train_dataset, val_dataset, test_dataset, class_names, class_weights = load_RWHAR_datasets(csv_file_name, config = 
                                                                                           {"window_size" : 50,
                                                                                            "overlap_ratio" : 0.5,})

print(f"Dataset size: Train {len(train_dataset)}, val {len(val_dataset)}, and test {len(test_dataset)}")
print("Class weights: \n", {c : round(w.item(), 2) for c, w in zip(class_names, class_weights)})

## 학습 (Training)

In [ ]:
def train_main(config):
    ## data and preprocessing settings
    csv_file_name = config['csv_file_name']
    num_worker = config.get('num_worker', 4)

    ## Hyper parameters
    batch_size = config['batch_size']
    learning_rate = config['learning_rate']
    start_epoch = config.get('start_epoch', 0)
    num_epochs = config['num_epochs']

    ## checkpoint setting
    checkpoint_save_interval = config.get('checkpoint_save_interval', 10)
    checkpoint_path = config.get('checkpoint_path', "checkpoints/checkpoint.pth")
    best_model_path = config.get('best_model_path', "checkpoints/best_model.pth")
    load_from_checkpoint = config.get('load_from_checkpoint', None)

    ## variables
    best_metric = 0

    device = "cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
    print(f"Using {device} device")

    train_dataset, val_dataset, test_dataset, class_names, class_weights = load_RWHAR_datasets(csv_file_name, config)
    num_classes = len(class_names)

    train_dataloader, val_dataloader, test_dataloader = create_dataloaders(train_dataset, val_dataset, test_dataset, device, 
                                                           batch_size = batch_size, num_worker = num_worker)

    model = get_model(model_name = config["model_name"], num_classes= num_classes, config = config).to(device)

    criterion = nn.CrossEntropyLoss(weight = class_weights).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=config['weight_decay'])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1) 

    if load_from_checkpoint:
        load_checkpoint_path = (best_model_path if load_from_checkpoint == "best" else checkpoint_path)
        start_epoch, best_metric = load_checkpoint(load_checkpoint_path, model, optimizer, scheduler, device)

    if config.get('test_mode', False):
        # Only evaluate on the test dataset
        print("Running test evaluation...")
        test_metric = evaluation_loop(model, device, test_dataloader, criterion, phase = "test")
        print(f"Test f1_macro: {test_metric}")

        return test_metric
    else:
        # Train and validate using train/val datasets
        for epoch in range(start_epoch, num_epochs):
            train_loop(model, device, train_dataloader, criterion, optimizer, epoch)
            val_metric = evaluation_loop(model, device, val_dataloader, criterion, epoch = epoch, phase = "validation")
            scheduler.step()

            if (epoch + 1) % checkpoint_save_interval == 0 or (epoch + 1) == num_epochs:
                is_best = val_metric > best_metric
                best_metric = max(val_metric, best_metric)
                save_checkpoint(checkpoint_path, model, optimizer, scheduler, epoch, best_metric, is_best, best_model_path)
                
            print(f"Epoch {epoch} validation f1_score {val_metric}")

        return best_metric

In [ ]:
config = {
    'csv_file_name': csv_file_name,
    "dataset": "RWHAR",
    'batch_size': 128,
    'learning_rate': 1e-4,
    'weight_decay' : 1e-6,
    'num_epochs': 10,

    "window_size" : 50,
    "overlap_ratio" : 0.5,

    # model architectures
    'model_name': 'DeepConvLSTM',
    'num_sensors' : 3,
    'num_filters': 64,
    'kernel_size': 11,
    'hidden_dim': 128,
    'num_layers': 1,
    'dropout_prob': 0.4,

    "checkpoint_save_interval" : 1,
    "checkpoint_path" : "checkpoints/checkpoint.pth",
    "best_model_path" : "checkpoints/best_model.pth",
    "load_from_checkpoint" : None,    # Options: "latest", "best", or None
}

In [ ]:
f1 = train_main(config)
print("Best f1 score: ", f1)

In [ ]:
config_testmode = {
    **config, 
    'test_mode': True, # True if evaluating only test set
    'load_from_checkpoint': 'best'
}

train_main(config_testmode)

### 모델 Validation (검증) 방법 개선을 통한 모델 일반화(generalization) 평가
모델의 일반화는 머신러닝 모델이 학습하지 새로운 데이터에 대해 얼마나 잘 예측하는지를 나타냅니다. 높은 일반화 능력은 모델이 실제 환경에서 효과적으로 동작할 수 있음을 의미합니다.

머신러닝/딥러닝 모델 개발과정에서 데이터셋을 적절히 나누어 모델이 얼마나 잘 일반화되는지 평가하는 것은 매우 중요합니다.

지금까지는 단순히 데이터를 훈련(train), 검증(validation), 테스트(test) 세트로 나누는 기본적인 검증 방법을 실습해 보았으며, 더 나은 검증 방법들은 다음과 같습니다:

1. 단순 훈련-검증 분할 (Train-Valid Split): 가장 기본적인 검증 방법으로, 데이터를 한 번의 훈련 세트와 검증 세트로 나눕니다. 이 방법은 구현이 간단하지만, 데이터 분할에 따라 결과가 크게 달라질 수 있다는 단점이 있습니다. 

2. K-겹 교차 검증 (K-Fold Cross-Validation): 데이터를 K개의 부분으로 나눈 후, 각 부분을 한 번씩 검증 세트로 사용하고 나머지는 훈련 세트로 사용하여 총 K번의 학습과 평가를 수행합니다. 이렇게 하면 모든 데이터가 검증에 활용되어 모델의 성능을 보다 안정적으로 평가할 수 있습니다.

3. 교차 참가자 검증 (Cross-Participant Cross-Validation): Leave-One-Subject-Out (LOSO)으로도 불리며, 참여자별로 데이터를 나누어 한 명의 참여자 데이터를 검증 세트로 사용하고 나머지 참여자의 데이터를 훈련 세트로 사용합니다. 이 과정을 모든 참여자에 대해 반복합니다. 이는 모델이 새로운 참여자에게도 일반화될 수 있는지 평가하는 데 효과적입니다.

교차 참가자 검증은 생물의학(biomedical) 데이터에서 Data leakage를 방지하고 모델의 일반화 성능을 높이는 데 특히 유용합니다.

<mark>Optional 실습</mark> `load_RWHAR_datasets`과 `train_main`함수를 수정하여 교차 참가자 검증을 수행하고 모델의 성능을 평가하세요.


### Lab을 마무리 짓기 전 저장된 checkpoint를 모두 지워 저장공간을 확보한다

In [ ]:
import shutil, os
if os.path.exists('checkpoints/'):
    shutil.rmtree('checkpoints/')

### With Label Smoothing
Now implement / try out HAR with different amounts of label smoothing.

In [ ]:
from torch import nn

class LabelSmoothingLoss(nn.Module):
    def __init__(self, smoothing=0.0):
        super(LabelSmoothingLoss, self).__init__()
        self.smoothing = smoothing

    def forward(self, prediction, target):
        assert 0 <= self.smoothing < 1
        neglog_softmaxPrediction = -prediction.log_softmax(dim=1)

        with torch.no_grad():
            smoothedLabels = self.smoothing / (prediction.size(1) - 1)* torch.ones_like(prediction)
            smoothedLabels.scatter_(1, target.data.unsqueeze(1), 1-self.smoothing)
        return torch.mean(torch.sum(smoothedLabels * neglog_softmaxPrediction, dim=1)) 

In [ ]:
criterion = LabelSmoothingLoss(0.4)

6.4 Visualization of the networks predictions on three classes

In [ ]:
## helper for extracting activations
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [ ]:
import matplotlib.pyplot as plt
## Visualization of the activations -- warning: This function expects the penultimate layer's name to be 'fc'
def visualizeActivations(classes, net, inputData, inputDataLabels, penultimateLayerName, lastLayerName):
    ## Extract the 'templates' to which the distances are computed 
    features = torch.zeros_like(getattr(net,lastLayerName).weight[0:3,:])
    for i in range(3):
        features[i,:] = -getattr(net,lastLayerName).weight[classes[i],:]
    
    ## Determine two orthogonal vectors that span the plane containing the three features
    V1 = features[0,:] - features[2,:]
    V1 = V1 / torch.norm(V1)
    V2 = features[1,:] - features[2,:]
    V2 = V2 - (V2.T@V1)*V1
    V2 = V2 / torch.norm(V2)
    
    net.eval()
    with torch.no_grad():
        ## Now compute the penultimate activations
        handle = getattr(net,penultimateLayerName).register_forward_hook(get_activation('penultimateActivations'))
        for i in range(3):
            output = net(inputData[inputDataLabels==classes[i],:,:])
            coord1 = activation['penultimateActivations']@V1[:,None]
            coord2 = activation['penultimateActivations']@V2[:,None]
            plt.plot(coord1.detach().cpu().numpy(), coord2.detach().cpu().numpy(), 'x')

        handle.remove()

In [ ]:
penultimateLayerName = 'dropout'
lastLayerName = 'fc'
classes = [0,1,2]


plt.title('with label smoothing')
visualizeActivations(classes, labelSmoothingNet, torch.from_numpy(X_train).to(config['gpu']),torch.from_numpy(y_train).to(config['gpu']), \
                    penultimateLayerName,lastLayerName) 
plt.show()


plt.title('without label smoothing')
visualizeActivations(classes, crossEntropyNet, torch.from_numpy(X_train).to(config['gpu']),torch.from_numpy(y_train).to(config['gpu']), \
                    penultimateLayerName,lastLayerName) 
plt.show()

6.6 MaxUp Implementation
This section refers to the second part - data augmentation and maxup training

6.6.1 MaxUp as a Class
As a suggestion, the Maxup class could consist of two functionalities: A forward pass that gets x and y as an input, and returns additional data as well as additional labels (increasing the amount of data by a factor of 'ntrials' or 'm' in the presentation). The second function is the adapted_loss that knows the structure of the data such that a reshaping is possible in order to have the 'ntrials'-many examples along one dimension, in which a maximum can be computed.

In [ ]:
## Implementation inspired by https://github.com/JonasGeiping/data-poisoning/
# see forest / data / mixing_data_augmentations.py

class Maxup(torch.nn.Module):
    """A meta-augmentation, returning the worst result from a range of augmentations.
    As in the orignal paper, https://arxiv.org/abs/2002.09024,
    """

    def __init__(self, given_data_augmentation, ntrials=4):
        """Initialize with a given data augmentation module."""
        super().__init__()
        self.augment = given_data_augmentation
        self.ntrials = ntrials
        self.max_criterion = torch.nn.CrossEntropyLoss(reduction='none')

    def forward(self, x, y):
        additional_x, additional_labels = [], []
        for trial in range(self.ntrials):
            x_out, y_out = self.augment(x, y)
            additional_x.append(x_out)
            additional_labels.append(y_out)

        additional_x = torch.cat(additional_x, dim=0)
        additional_labels = torch.cat(additional_labels, dim=0)
        
        return additional_x, additional_labels


    def maxup_loss(self, outputs, extra_labels):
        """Compute loss. Here the loss is computed as worst-case estimate over the trials."""
        batch_size = outputs.shape[0] // self.ntrials
        correct_preds = (torch.argmax(outputs.data, dim=1) == extra_labels).sum().item() / self.ntrials
        stacked_loss = self.max_criterion(outputs, extra_labels).view(batch_size, self.ntrials, -1)
        loss = stacked_loss.max(dim=1)[0].mean()
        
        return loss, correct_preds
    
def myNoiseAdditionAugmenter(x,y):
    sigma = 0.5
    return x + sigma*torch.randn_like(x), y

Data Augmentation

adding noise to the signals, random scaling, and jittering. Explain how these techniques can improve model generalization.

adding noise, rotating accelerometer signals
time warping, permutation. 

In [ ]:
import torch
from torch.utils.data import DataLoader

# initialize your DeepConvLSTM object 
network = DeepConvLSTM(config)

# sends network to the GPU and sets it to training mode
network.to(config['gpu'])
network.train()

# initialize the optimizer and loss
optimizer = torch.optim.Adam(network.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
maxup = Maxup(myNoiseAdditionAugmenter, ntrials=4)


# initialize training and validation dataset, define DataLoaders
dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_valid).float(), torch.from_numpy(y_valid))
valloader = DataLoader(dataset, batch_size=config['batch_size'], shuffle=True)
dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
trainloader = DataLoader(dataset, batch_size=config['batch_size'], shuffle=True)


# define your training loop; iterates over the number of epochs
for e in range(config['epochs']):
    counter = 1

    # iterate over the trainloader object (it'll return batches which you can use)
    for i, (x, y) in enumerate(trainloader):
        # sends batch x and y to the GPU
        inputs, targets = x.to(config['gpu']), y.to(config['gpu'])
        
        optimizer.zero_grad()

        # Increase the inputs via data augmentation
        inputs,targets = maxup(inputs,targets)
        
        # send inputs through network to get predictions
        train_output = network(inputs)
        

        # calculates loss
        loss = maxup.maxup_loss(train_output, targets.long())[0]

        # backprogate your computed loss through the network
        # use the .backward() and .step() function on your loss and optimizer
        loss.backward()
        optimizer.step()


        # prints out every 100 batches information about the current loss and time per batch
        if counter % 80 == 0:
            print('| epoch {:3d} | current minibatch train loss {:5.2f}'.format(e, loss.item()))
        
        counter += 1